<a href="https://colab.research.google.com/github/rishurnjan/Fine_Tuning/blob/main/GPT_2_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:

from datasets import load_dataset
import pandas as pd
dataset= load_dataset("mteb/tweet_sentiment_extraction")
df=pd.DataFrame(dataset['train'])
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/3.63M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/465k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative


In [3]:
from transformers import GPT2Tokenizer
tokenizer= GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token =tokenizer.eos_token
def tokenize_function(examples):
  return tokenizer(examples['text'],padding="max_length", truncation=True)

tokenized_dataset= dataset.map(tokenize_function,batched=True)
tokenized_dataset

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 3534
    })
})

In [4]:
small_train_dataset= tokenized_dataset['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset= tokenized_dataset['test'].shuffle(seed=42).select(range(1000))

In [5]:
from transformers import GPT2ForSequenceClassification
model= GPT2ForSequenceClassification.from_pretrained('gpt2',num_labels=3)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00


In [7]:
import evaluate
import numpy as np

metric=evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels=eval_pred
  predictions =np.argmax(logits,axis=-1 )
  return metric.compute(predictions=predictions,references=labels)

In [8]:
from transformers import TrainingArguments, Trainer #Fixed typo in TrainingArguments

training_args = TrainingArguments(
   output_dir="test_trainer",
   #evaluation_strategy="epoch",
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   gradient_accumulation_steps=4
   )


trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,

)

trainer.train()

Step,Training Loss
500,0.916800


TrainOutput(global_step=750, training_loss=0.7756990051269531, metrics={'train_runtime': 785.8899, 'train_samples_per_second': 3.817, 'train_steps_per_second': 0.954, 'total_flos': 1567794659328000.0, 'train_loss': 0.7756990051269531, 'epoch': 3.0})

In [9]:
import evaluate

trainer.evaluate()


{'eval_loss': 1.025317907333374,
 'eval_accuracy': 0.73,
 'eval_runtime': 72.9479,
 'eval_samples_per_second': 13.708,
 'eval_steps_per_second': 13.708,
 'epoch': 3.0}